# 1 Intro

This notebook takes you through some basic data manipulation and analysis tasks by utilizing a few popular Python data science packages. In particular, we will use the following 4 packages.

1. [pandas](https://pandas.pydata.org/), a library for data processing. It mainly operates on 2D tables with columns and rows called dataframes.
2. [yfinance](https://ranaroussi.github.io/yfinance/index.html), a library to download market data from Yahoo Finance.
3. [matplotlib](https://matplotlib.org/), a library for plotting charts.
4. [scikit-learn](https://scikit-learn.org/stable/index.html), a library for machine learning.

The purpose of this notebook is to get you started with the basics of data processing and modeling. Refer to the "Resources" section in the workshop website, and you will find more learning materials to advance your Python data science skill.

# 2 Data

Before we can use third party modules (i.e., packages or libraries), we need to import them. Normally, we would first need to install the modules. However, since we are running the code in Google Colab, and Google Colab has already installed many commonly used Python module for us, we can skip the installation step here.

(You can use the command `!pip list` to list all packages that has been installed in Google Colab. The `!` in the command means that `pip list` is a terminal utility command rather than Python code.)

In [ ]:
# import the modules needed for this notebook
# we will import sklearn later
import pandas as pd
import yfinance as yf

import matplotlib.pyplot as plt

The [yfinance](https://ranaroussi.github.io/yfinance/index.html) package can retrieve data from Yahoo Finance in many different ways. Here, we simply download a daily "SPY" series using its [`download()`](https://ranaroussi.github.io/yfinance/reference/api/yfinance.download.html#yfinance.download) function. The function returns a pandas dataframe.

In our example, we set the `download()` function's `auto_adjust` parameter to `False` so it returns explicitly both closing price and adjusted closing price. By default, `auto_adjust=True` and the closing price is automatically/implicitly adjusted and returned as closing price.

We also set the `multi_level_index` to `False` so that the columns of the returning dataframe have only one level instead of two, for example, the adjusted closing price column is `Adj Close` instead of `(Adj Close, SPY)`. This simplifies the returned dataframe and works well for our case since we only plan to download data for a single ticker SPY.

In [ ]:
# downlad SPY time series from Yahoo Finance
df = yf.download("SPY", start="2023-01-01", end="2024-06-30", auto_adjust=False, multi_level_index=False)

In [ ]:
# take a look at first few rows
df.head()

A pandas dataframe consists of an index and columns. The index of a dataframe is a series of labels that identify each row. A column is a series of values identified by a column head/name/variable. (Both index and column can have mulitple levels in more complicated cases.)

In [ ]:
# the Date column is not really a column but an index, in particular, DatetimeIndex
# the columns are Adj Close, Close, High, Low, Open, Volume
df.info()

In [ ]:
# display the index
df.index

In [ ]:
# display column names
df.columns

In [ ]:
# descriptive statistics
df.describe()

`.loc` and `.iloc` are methods associated with a dataframe for indexing, slicing and filtering.

In [ ]:
# locate rows based on a condition and select only 'Adj Close' column
df.loc[df.index<='2023-01-05', ['Adj Close']]

In [ ]:
# locate rows and select columns using integer positions
# :5 means from start to the fifth row (excluding the fifth row), same as 0:5
# -1 means the last column
df.iloc[:5, -1]

Among all the columns, we will only use two, `Volume` and `Adj Close`.

In [ ]:
# select only two columns, ['Adj Close', 'Volume']
df = df[['Adj Close', 'Volume']]

We will use matplotlib to display adjusted closing prices over time in a line plot.

In [ ]:
# create figure and axes
fig, ax = plt.subplots(figsize=(10, 5))

# line plot
ax.plot(df['Adj Close'])

# set the title
ax.set_title('Daily Adjusted Closing Price of SPY (USD)')

# display plot (optional in the notebook setting)
plt.show()

## Exercise

Plot a line chart for SPY daily volumes in 2024.

In [ ]:
# your code here


# 3 Simple Analysis

The DatetimeIndex gives us more ways of extracting information from it. For example, we can extract year, month and weekday information for each date and create new dataframe columns for them.


In [ ]:
# create year, month and weekday columns
df['year']=df.index.year
df['month']=df.index.month
df['weekday']=df.index.weekday
df.head()

Methods such as `.groupby` can be used for data aggregation.

In [ ]:
# mean adjusted closing price by year and month
df.groupby(['year', 'month'])['Adj Close'].mean()

In [ ]:
# since the data is sorted by the DatetimeIndex, we can also retrieve the
# end of month adjusted closing price
df.groupby(['year', 'month'])['Adj Close'].last()

Let's calculate the daily returns.

In [ ]:
# since the data is sorted by the DatetimeIndex, shift(1) gives previous
# trading day's adjusted closing price
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html
df['return'] = (df['Adj Close'] / df['Adj Close'].shift(1) - 1) * 100
df.head()

In [ ]:
# plot daily returns
# create figure and axes
fig, ax = plt.subplots(figsize=(10, 5))

# line plot
ax.plot(df['return'])

# set the title
ax.set_title('Daily Return of SPY (%)')

# display plot (optional in the notebook setting)
plt.show()

Calculating cumulative returns is also easy.

In [ ]:
# cumprod() calculates cumulative product
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.cumprod.html
df['cum_return'] = ((df['Adj Close'] / df['Adj Close'].shift(1)).cumprod() - 1) * 100
df.head()

In [ ]:
# plot cumulative returns

# create figure and axes
fig, ax = plt.subplots(figsize=(10, 5))

# line plot
ax.plot(df['cum_return'])

# set the title
ax.set_title('Cumulative Return of SPY (%)')

# display plot (optional in the notebook setting)
plt.show()

A **(simple) moving average (MA)** is a widely used indicator in technical analysis that helps smooth out price actions by filtering out the “noise” from random short-term price fluctuations. It is a trend-following, or lagging, indicator because it is based on past prices.

Source: https://www.investopedia.com/terms/m/movingaverage.asp

In [ ]:
# rolling() provides rolling window calculation
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html
df['ma_50d'] = df['Adj Close'].rolling(window=50).mean()
df['ma_100d'] = df['Adj Close'].rolling(window=100).mean()
df.tail()

Now, let's plot the adjusted closing price along with the two moving averages we calculated in one plot.

In [ ]:
# create figure and axes
fig, ax = plt.subplots(figsize=(10, 5))

# line plot
ax.plot(df['Adj Close'], label="Adjusted Closing Price")
ax.plot(df['ma_50d'], label="50-day Moving Average")
ax.plot(df['ma_100d'], label="100-day Moving Average")

# set the title
ax.set_title('Daily Adjusted Closing Price of SPY (USD)')

# add legend (based on line labels)
ax.legend()

# display plot (optional in the notebook setting)
plt.show()

## Exercise

Calculate weekly volatilities (5-day window) for the SPY series, and plot the result in a line chart.

**Volatility** is a statistical measure of the dispersion of returns for a given security or market index. In most cases, the higher the volatility, the riskier the security. Volatility is often measured as either the standard deviation or variance between returns from that same security or market index.

Source: https://www.investopedia.com/terms/v/volatility.asp

Hint: consider the `.rolling()` and `.std()` methods.

In [ ]:
# your code here
# hint: consider the .rolling() and .std() method.


# 4 Linear regression

We can build a model to predict future stock prices and see how it performs by comparing it against the actual data. It's often a good idea to start with a simple model as its performance can serve as a benchmark for more complicated models. Thus, we start with a linear regression model. We will predict the next day SPY price with today's price, volume and 50-day moving average.

Let's prepare the data for this task.

In [ ]:
# create a new column y_price holding next trading day's adjusted closing price
df['y_price'] = df['Adj Close'].shift(-1)
df.head()

In [ ]:
# create a new dataframe that only contains columns needed for the regression analysis
df_reg = df[['Adj Close', 'Volume','ma_50d', 'y_price']].copy()

# drop all rows with missing value (mostly due to the 50-day MA column)
df_reg.dropna(inplace=True)

# let X be today's closing price, volume, 50-day MA
X = df_reg[['Adj Close', 'Volume','ma_50d']]

# let y be the next day price to predict
# that is, we predict price using 1-day lagged information
y = df_reg['y_price']

We split the full dataset into training and test sets. We use the training set to train/fit the model (i.e. to estimate the coefficients of the linear regression model), and the test dataset to evaluate the fitted model's performance.

We must respect the time dimension of our time series data when splitting the data into training and test sets. After all, we cannot train on future data/information and predict the price on past data. Therefore, we use the data in 2023 for training and the data in 2024 for testing.

In [ ]:
# use data in 2023 for training
X_train = X[X.index < '2024-01-01']
y_train = y[y.index < '2024-01-01']

# use data in 2024 for testing
X_test = X[X.index >= '2024-01-01']
y_test = y[y.index >= '2024-01-01']

We use the [`LinearRegression()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) function in the [sklearn](https://scikit-learn.org/stable/) package for the linear regression task.

(There are other options. For example, the scientific computing package [scipy](https://scipy.org/) offers a linear regression function [`linregress()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html). The statistics package [statsmodels](https://www.statsmodels.org/stable/index.html) also supports the [`OLS()`](https://www.statsmodels.org/stable/regression.html) model, Ordinary Least Square model.)

After the regression, we evaluate the fitted model on the test set using the MSE (Mean Squared Error) metric. We also plot the true and predicted prices.

In [ ]:
# run regression

# load the relevant sklearn module
from sklearn.linear_model import LinearRegression

# initialize a regression model
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
lr = LinearRegression()

# train/fit on training data
lr.fit(X_train, y_train)

# predict on test data
y_test_pred = lr.predict(X_test)

Note that the `.predict()` method returns a numpy array. [Numpy](https://numpy.org/) is yet another popular Python data science library. It makes operations on arrays/matrices easy to handle.

In [ ]:
# note the prediction object obtained is a numpy array
type(y_test_pred)

In [ ]:
# note the prediction object obtained is a numpy array
y_test_pred

In [ ]:
# calculate test MSE (mean squared error)

# load the relevant sklearn module
from sklearn.metrics import mean_squared_error

# calculate the MSE between the true prices in the test data and the predicted prices
mse = mean_squared_error(y_test, y_test_pred)
print(f"Mean Squared Error: {mse}")

In [ ]:
# prepare data for a true vs predicted price plot

# create a true vs pred dataframe based on the resulting prediction numpy array and the test data DatetimeIndex
df_y_true_vs_pred = pd.DataFrame(y_test_pred, index=y_test.index, columns=[["pred"]])

df_y_true_vs_pred["true"] = y_test

# however, since we are predicting next day's price, we need to shift(1) to align
# the date and the true and predicted value
df_y_true_vs_pred = df_y_true_vs_pred.shift(1)
df_y_true_vs_pred.head()

In [ ]:
# plot the prediction and true price for comparison

# create figure and axes
fig, ax = plt.subplots(figsize=(10, 5))

# line plot
ax.plot(df_y_true_vs_pred["true"], label="True Price")
ax.plot(df_y_true_vs_pred["pred"], label="Predicted Price")

# set the title
ax.set_title('Regression Analysis: True vs Predicted Price')

# add legend (based on line labels)
ax.legend()

# display plot (optional in the notebook setting)
plt.show()

## Exercise

Can you improve the test MSE by modifying the linear regression model, for example, adding new X variables?

In [ ]:
# your code here


# Appendix

We can also perform the linear regression analysis using the statistics package [statsmodels](https://www.statsmodels.org/stable/index.html). The package supports the [`OLS()`](https://www.statsmodels.org/stable/regression.html) model (Ordinary Least Square model is just another name for linear regression). It also offers an easy way to write [regression formula](https://www.statsmodels.org/dev/example_formulas.html), and produces a nice regression report. This regression report is especially useful for causal analysis, where you care about statistical inference in the regression (e.g., confidence intervals or hypothesis tests for the estimated coefficients).



In [ ]:
import statsmodels.formula.api as smf

# only use 2023 data for training
df_reg_train = df_reg.loc[df_reg.index < '2024-01-01']

# create an OLS model
# note the formula; because the column name 'Adj Close' contains a space, we need to wrap it in a Q() function
ols_model = smf.ols(formula="y_price ~ Q('Adj Close') + Volume + ma_50d", data=df_reg_train)

# fit the OLS model
result = ols_model.fit()

# print the regression report
print(result.summary())